<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 16


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

<h2 style="color:DodgerBlue">Описание проекта:</h2>

Создать базовый класс PaymentMethod в C#, который будет представлять
различные способы оплаты. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма..
Требования к базовому классу PaymentMethod:
• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты
(MethodName), Минимальная сумма (MinAmount).
• Методы:
o ProcessPayment(decimal amount): метод для обработки платежа
указанной суммы.
o CheckMinimumAmount(decimal amount): метод для проверки
минимальной суммы платежа.
o GetPaymentDetails(): метод для получения деталей способа оплаты.
Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные
атрибуты, такие как URL платежной системы (PaymentUrl).
Метод ProcessPayment() должен быть переопределен для включения URL
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные
атрибуты, такие как Банковские данные (BankData).
Метод CheckMinimumAmount() должен быть переопределен для проверки
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать
дополнительные атрибуты, такие как Место выдачи наличных
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен
для отображения места выдачи наличных. 

#### Дополнительное задание
Добавьте к сущестующим классам конструктора классов с использованием гетторов и сетторов и реализуйте взаимодействие объектов между собой

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [3]:
using System;
using System.Collections.Generic;

// Интерфейс оплаты
public interface IPayment
{
    string GetPaymentDetails();
    void ProcessPayment(decimal amount);
}

// Делегаты для событий
public delegate void PaymentAction(string message);

// Базовый класс
public abstract class PaymentMethod : IPayment
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    public string Currency { get; set; } 
    public string Description { get; set; }
    public bool IsActive { get; set; } = true;

    public event PaymentAction OnProcess;
    public event PaymentAction OnCheck;

    protected void RaiseProcess(string message) => OnProcess?.Invoke(message);
    protected void RaiseCheck(string message) => OnCheck?.Invoke(message);

    protected PaymentMethod(int id, string name, decimal minAmount, string currency, string description)
    {
        PaymentMethodId = id;
        MethodName = name;
        MinAmount = minAmount;
        Currency = currency;
        Description = description;
    }

    public virtual void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            RaiseProcess($"Платеж {amount} {Currency} обработан методом {MethodName}.");
        }
        else
        {
            RaiseProcess($"Сумма {amount} {Currency} меньше минимальной ({MinAmount} {Currency}).");
        }
    }

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        bool result = amount >= MinAmount;
        RaiseCheck(result
            ? $"Сумма {amount} {Currency} удовлетворяет минимальной ({MinAmount} {Currency})."
            : $"Сумма {amount} {Currency} меньше минимальной ({MinAmount} {Currency}).");
        return result;
    }

    public virtual string GetPaymentDetails()
    {
        return $"ID: {PaymentMethodId}, Метод: {MethodName}, Мин. сумма: {MinAmount} {Currency}, Активен: {IsActive}, Описание: {Description}";
    }
}

// Онлайн оплата
public class OnlinePayment : PaymentMethod
{
    public string PaymentUrl { get; set; }
    public string Provider { get; set; }
    public int TimeoutSeconds { get; set; }

    public OnlinePayment(int id, string name, decimal minAmount, string currency, string description, string url, string provider, int timeout)
        : base(id, name, minAmount, currency, description)
    {
        PaymentUrl = url;
        Provider = provider;
        TimeoutSeconds = timeout;
    }

    public override void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            RaiseProcess($"Онлайн-платеж {amount} {Currency} через {Provider} ({PaymentUrl}) успешно выполнен.");
        }
        else
        {
            RaiseProcess($"Невозможно обработать онлайн-платеж {amount} {Currency}. Минимальная сумма: {MinAmount} {Currency}.");
        }
    }

    public override string GetPaymentDetails()
    {
        return base.GetPaymentDetails() + $", URL: {PaymentUrl}, Провайдер: {Provider}, Таймаут: {TimeoutSeconds}s";
    }
}

// Банковский перевод
public class BankTransfer : PaymentMethod
{
    public string BankData { get; set; }
    public decimal Commission { get; set; }
    public string BankName { get; set; } 

    public BankTransfer(int id, string name, decimal minAmount, string currency, string description, string bankData, decimal commission, string bankName)
        : base(id, name, minAmount, currency, description)
    {
        BankData = bankData;
        Commission = commission;
        BankName = bankName;
    }

    public override bool CheckMinimumAmount(decimal amount)
    {
        decimal totalAmount = amount + Commission;
        bool result = totalAmount >= MinAmount;
        RaiseCheck(result
            ? $"Сумма {amount} {Currency} с комиссией {Commission} {Currency} удовлетворяет минимальной ({MinAmount} {Currency})."
            : $"Сумма {amount} {Currency} с комиссией {Commission} {Currency} меньше минимальной ({MinAmount} {Currency}).");
        return result;
    }

    public override string GetPaymentDetails()
    {
        return base.GetPaymentDetails() + $", Банк: {BankName}, Данные банка: {BankData}, Комиссия: {Commission} {Currency}";
    }
}

// Наличные
public class CashPayment : PaymentMethod
{
    public string CashPickupPoint { get; set; }
    public string City { get; set; } 
    public int WorkingHours { get; set; }

    public CashPayment(int id, string name, decimal minAmount, string currency, string description, string pickupPoint, string city, int hours)
        : base(id, name, minAmount, currency, description)
    {
        CashPickupPoint = pickupPoint;
        City = city;
        WorkingHours = hours;
    }

    public override string GetPaymentDetails()
    {
        return base.GetPaymentDetails() + $", Пункт выдачи: {CashPickupPoint}, Город: {City}, Часы работы: {WorkingHours}";
    }
}

// Generic класс для управления платежами
public class PaymentManager<T> where T : PaymentMethod
{
    private List<T> payments = new List<T>();

    public event PaymentAction OnAddPayment;

    public void AddPayment(T payment)
    {
        payments.Add(payment);
        OnAddPayment?.Invoke($"Добавлен метод оплаты: {payment.MethodName}");
    }

    public void ShowAllPayments()
    {
        foreach (var p in payments)
            Console.WriteLine(p.GetPaymentDetails());
    }
}

// Проверка работы
var online = new OnlinePayment(1, "PayPal", 10, "USD", "Онлайн ооплата", "https://paypal.com", "PayPal Inc.", 30);
var bank = new BankTransfer(2, "Сбербанк", 100, "RUB", "Банковский перевод", "Счет №123456", 5, "Сбербанк");
var cash = new CashPayment(3, "Наличные", 50, "RUB", "Оплата наличными", "Главная площадь, 1", "Москва", 9);

online.OnProcess += msg => Console.WriteLine(msg);
bank.OnCheck += msg => Console.WriteLine(msg);
cash.OnProcess += msg => Console.WriteLine(msg);

online.ProcessPayment(15);
bank.ProcessPayment(95); 
cash.ProcessPayment(60);

var manager = new PaymentManager<PaymentMethod>();
manager.OnAddPayment += msg => Console.WriteLine(msg);
manager.AddPayment(online);
manager.AddPayment(bank);
manager.AddPayment(cash);
manager.ShowAllPayments();


Онлайн-платеж 15 USD через PayPal Inc. (https://paypal.com) успешно выполнен.
Сумма 95 RUB с комиссией 5 RUB удовлетворяет минимальной (100 RUB).
Платеж 60 RUB обработан методом Наличные.
Добавлен метод оплаты: PayPal
Добавлен метод оплаты: Сбербанк
Добавлен метод оплаты: Наличные
ID: 1, Метод: PayPal, Мин. сумма: 10 USD, Активен: True, Описание: Онлайн ооплата, URL: https://paypal.com, Провайдер: PayPal Inc., Таймаут: 30s
ID: 2, Метод: Сбербанк, Мин. сумма: 100 RUB, Активен: True, Описание: Банковский перевод, Банк: Сбербанк, Данные банка: Счет №123456, Комиссия: 5 RUB
ID: 3, Метод: Наличные, Мин. сумма: 50 RUB, Активен: True, Описание: Оплата наличными, Пункт выдачи: Главная площадь, 1, Город: Москва, Часы работы: 9
